### Investigating the **RAW MBO Data** (includes all order book levels and all futures contracts at a given time)

In [ ]:
import databento as db
import pandas as pd
from pathlib import Path
from IPython.display import display

# Constants 
CORN_MONTH_ORDER = ["H", "K", "N", "U", "Z"]  # March, May, July, Sep, Dec
ACTION_CODES = ["T", "A", "R", "C", "F", "N", "M"]
ACTION_LABELS = ["trades", "adds", "clears", "cancels", "fills", "none", "modify"]


def show_info_for_single_day_data(data_path: str) -> None:
    """
    Display a quick summary of a single day's corn futures market data.
    
    - sample rows and column types
    - action breakdown table (trades, adds, cancels, etc.)
    - symbol counts
    - front month contract identification
    
    """
    
    # Check if file exists
    file_path = Path(data_path)
    if not file_path.exists():
        print(f" file NOT found: {data_path}")
        print(f" > This day does not exist in the dataset. Check the filename.")
        return None
    
    # Load data
    corn_dbn_file = db.DBNStore.from_file(data_path)
    corn_book_df = corn_dbn_file.to_df()

    # Display sample and column types
    display(corn_book_df.head(6))
    print(corn_book_df.dtypes)
    
    # Action stats (one pass over the action column for efficiency)
    action_counts = corn_book_df["action"].value_counts()
    action_counts_dict = {code: int(action_counts.get(code, 0)) for code in ACTION_CODES}
    
    # Summary table
    summary_df = pd.DataFrame({
        "metric": ACTION_LABELS,
        "action_code": ACTION_CODES,
        "count": list(action_counts_dict.values()),
    })
    
    display(summary_df)
    
    # Symbol counts
    symbol_counts = corn_book_df["symbol"].value_counts()
    symbol_counts_df = pd.DataFrame({
        "symbol": symbol_counts.index,
        "count": symbol_counts.values,
    })
    display(symbol_counts_df)
    
    # Find front-month contract (earliest expiration among base contracts)
    base_contracts = symbol_counts_df[symbol_counts_df["symbol"].str.len() <= 4].copy()
    
    
    # Symbol format: ZC + month_letter + year_digit (e.g., ZCH6 = March 2026)
    base_contracts.loc[:, "month_letter"] = base_contracts["symbol"].str[2]
    base_contracts.loc[:, "year_digit"] = base_contracts["symbol"].str[3].astype(int)
    base_contracts.loc[:, "month_index"] = base_contracts["month_letter"].map(
        {letter: idx for idx, letter in enumerate(CORN_MONTH_ORDER)}
    )
    
    # Sort by year, then month
    base_contracts_sorted = base_contracts.sort_values(["year_digit", "month_index"])
    
    front_month = base_contracts_sorted.iloc[0]["symbol"]
    print(f"\nfront month contract is: {front_month}")
    
    return None


In [69]:
data_path:str = "../data/raw/glbx-mdp3-20260204.mbo.dbn" # 4th Feb 2026
show_info_for_single_day_data(data_path)

,ts_event,rtype,publisher_id,instrument_id,action,side,price,size,channel_id,order_id,flags,ts_in_delta,sequence,symbol
ts_recv,,,,,,,,,,,,,,
2026-02-04 00:00:00+00:00,2026-02-01 13:42:54.645765203+00:00,160,1,42030576,R,N,NaN,0,15,0,40,0,0,ZCN9-ZCZ9
2026-02-04 00:00:00+00:00,2026-02-01 13:42:54.645765203+00:00,160,1,42030576,A,B,10.50,2,15,7012604031888,40,0,6114,ZCN9-ZCZ9
2026-02-04 00:00:00+00:00,2026-02-01 13:42:54.645765203+00:00,160,1,42030576,A,B,10.25,2,15,7012570445318,40,0,6114,ZCN9-ZCZ9
2026-02-04 00:00:00+00:00,2026-02-01 13:42:54.645765203+00:00,160,1,42030576,A,B,5.25,2,15,7012570445374,40,0,6114,ZCN9-ZCZ9
2026-02-04 00:00:00+00:00,2026-02-01 13:42:54.645765203+00:00,160,1,42030576,A,B,5.00,2,15,7012570445596,40,0,6114,ZCN9-ZCZ9
2026-02-04 00:00:00+00:00,2026-02-01 13:42:54.645765203+00:00,160,1,42030576,A,B,0.25,2,15,7012570445531,40,0,6114,ZCN9-ZCZ9


ts_event         datetime64[ns, UTC]
rtype                          uint8
publisher_id                  uint16
instrument_id                 uint32
action                           str
side                             str
price                        float64
size                          uint32
channel_id                     uint8
order_id                      uint64
flags                          uint8
ts_in_delta                    int32
sequence                      uint32
symbol                           str
dtype: object


,metric,action_code,count
0,trades,T,47367
1,adds,A,653616
2,clears,R,95
3,cancels,C,613169
4,fills,F,114225
5,none,N,0
6,modify,M,423210


,symbol,count
0,ZCH6,496957
1,ZCK6,224643
2,ZCN6,136813
3,ZCZ6,130230
4,ZCU6,121553
...,...,...
90,ZC:CF N7U7Z7H8,2
91,ZCU8,2
92,ZCN7-ZCN8,2
93,ZC:CF K6N6U6Z6,2



front month contract is: ZCH6


In [70]:
data_path:str = "../data/raw/glbx-mdp3-20260205.mbo.dbn" # 5th Feb 2026
show_info_for_single_day_data(data_path)

,ts_event,rtype,publisher_id,instrument_id,action,side,price,size,channel_id,order_id,flags,ts_in_delta,sequence,symbol
ts_recv,,,,,,,,,,,,,,
2026-02-05 00:00:00+00:00,2026-02-01 13:42:54.645765203+00:00,160,1,42030576,R,N,NaN,0,15,0,40,0,0,ZCN9-ZCZ9
2026-02-05 00:00:00+00:00,2026-02-01 13:42:54.645765203+00:00,160,1,42030576,A,B,10.50,2,15,7012604031888,40,0,6114,ZCN9-ZCZ9
2026-02-05 00:00:00+00:00,2026-02-01 13:42:54.645765203+00:00,160,1,42030576,A,B,10.25,2,15,7012570445318,40,0,6114,ZCN9-ZCZ9
2026-02-05 00:00:00+00:00,2026-02-01 13:42:54.645765203+00:00,160,1,42030576,A,B,5.25,2,15,7012570445374,40,0,6114,ZCN9-ZCZ9
2026-02-05 00:00:00+00:00,2026-02-01 13:42:54.645765203+00:00,160,1,42030576,A,B,5.00,2,15,7012570445596,40,0,6114,ZCN9-ZCZ9
2026-02-05 00:00:00+00:00,2026-02-01 13:42:54.645765203+00:00,160,1,42030576,A,B,0.25,2,15,7012570445531,40,0,6114,ZCN9-ZCZ9


ts_event         datetime64[ns, UTC]
rtype                          uint8
publisher_id                  uint16
instrument_id                 uint32
action                           str
side                             str
price                        float64
size                          uint32
channel_id                     uint8
order_id                      uint64
flags                          uint8
ts_in_delta                    int32
sequence                      uint32
symbol                           str
dtype: object


,metric,action_code,count
0,trades,T,39636
1,adds,A,398714
2,clears,R,96
3,cancels,C,358047
4,fills,F,102070
5,none,N,0
6,modify,M,269175


,symbol,count
0,ZCH6,308229
1,ZCK6,122332
2,ZCN6,86570
3,ZCZ6,78968
4,ZCU6,76612
...,...,...
91,ZC:CF K6N6U6Z6,2
92,ZCH7-ZCK8,2
93,ZC:CF N7U7Z7H8,2
94,ZCU8,2



front month contract is: ZCH6


In [71]:
data_path:str = "../data/raw/glbx-mdp3-20240204.mbo.dbn" # 4th Feb 2024
show_info_for_single_day_data(data_path)

,ts_event,rtype,publisher_id,instrument_id,action,side,price,size,channel_id,order_id,flags,ts_in_delta,sequence,symbol
ts_recv,,,,,,,,,,,,,,
2024-02-04 12:03:28.853292028+00:00,2024-02-04 12:03:28.737000+00:00,160,1,229776,R,N,NaN,0,15,0,8,0,0,ZCN5
2024-02-04 12:03:28.868594683+00:00,2024-02-04 12:03:28.737000+00:00,160,1,487769,R,N,NaN,0,15,0,8,0,0,ZCZ4
2024-02-04 12:03:28.894811579+00:00,2024-02-04 12:03:28.787000+00:00,160,1,42273528,R,N,NaN,0,15,0,8,0,0,ZCK6
2024-02-04 12:03:28.895665799+00:00,2024-02-04 12:03:28.787000+00:00,160,1,495552,R,N,NaN,0,15,0,8,0,0,ZCN6
2024-02-04 12:03:28.913755883+00:00,2024-02-04 12:03:28.837000+00:00,160,1,272731,R,N,NaN,0,15,0,8,0,0,ZCU5
2024-02-04 12:03:28.922265450+00:00,2024-02-04 12:03:28.837000+00:00,160,1,42261906,R,N,NaN,0,15,0,8,0,0,ZCZ7


ts_event         datetime64[ns, UTC]
rtype                          uint8
publisher_id                  uint16
instrument_id                 uint32
action                           str
side                             str
price                        float64
size                          uint32
channel_id                     uint8
order_id                      uint64
flags                          uint8
ts_in_delta                    int32
sequence                      uint32
symbol                           str
dtype: object


,metric,action_code,count
0,trades,T,0
1,adds,A,35646
2,clears,R,168
3,cancels,C,128
4,fills,F,0
5,none,N,0
6,modify,M,22


,symbol,count
0,ZCH4,10948
1,ZCZ4,7809
2,ZCH4-ZCK4,2223
3,ZCH4-ZCN4,1408
4,ZCK4,1349
...,...,...
163,ZCH5-ZCH6,1
164,ZCK4-ZCK5,1
165,ZCN4-ZCZ7,1
166,ZCU4-ZCU5,1



front month contract is: ZCH4


In [65]:
data_path:str = "../data/raw/glbx-mdp3-202512030.mbo.dbn" # 5th Dec 2025
show_info_for_single_day_data(data_path)

 file NOT found: ../data/raw/glbx-mdp3-202512030.mbo.dbn
 > This day does not exist in the dataset. Check the filename


### Investigating the **NEW Processed Trades Data** (includes only trades, so not outstanding bid/offer, and only to the 10th order level and only for the front future contract at that given time.)

In [74]:
import pandas as pd
processed_trades_data = pd.read_parquet("../data/processed/front_month_trades.parquet")
print(f"Shape: {processed_trades_data.shape}")        # (number_of_trades, number_of_columns)
print(f"Columns: {processed_trades_data.columns.tolist()}")
print(f"Date range: {processed_trades_data['date'].min()} to {processed_trades_data['date'].max()}")
print(f"Contracts: {sorted(processed_trades_data['symbol'].unique())}")
processed_trades_data.head(30)

Shape: (5187471, 6)
Columns: ['ts_event', 'price', 'size', 'side', 'symbol', 'date']
Date range: 20240205 to 20260205
Contracts: ['ZCH4', 'ZCH5', 'ZCH6', 'ZCK4', 'ZCK5', 'ZCN4', 'ZCN5', 'ZCU4', 'ZCU5', 'ZCZ4', 'ZCZ5']


,ts_event,price,size,side,symbol,date
0,2024-02-05 01:00:00+00:00,442.75,172,N,ZCH4,20240205
1,2024-02-05 01:00:00.002065937+00:00,442.75,1,B,ZCH4,20240205
2,2024-02-05 01:00:00.002833689+00:00,442.75,1,N,ZCH4,20240205
3,2024-02-05 01:00:00.003107105+00:00,442.50,6,A,ZCH4,20240205
4,2024-02-05 01:00:00.003107105+00:00,442.25,4,A,ZCH4,20240205
5,2024-02-05 01:00:00.003327971+00:00,442.25,5,A,ZCH4,20240205
6,2024-02-05 01:00:00.139183643+00:00,442.50,2,B,ZCH4,20240205
7,2024-02-05 01:00:00.140451931+00:00,442.50,1,N,ZCH4,20240205
8,2024-02-05 01:00:00.159838879+00:00,442.50,1,B,ZCH4,20240205
9,2024-02-05 01:00:00.170269221+00:00,442.25,14,A,ZCH4,20240205
